## Machine Learning Assignment #1 - Image Classification

### Implement the missing parts to train an image classification model:
- configuration of the model.
- loss function for training the model.

### Implementing an approach to mitigate and eliminate noise inherent in the provided data:
- there is no fixed or predefined answer.
- include brief comments in the code to explain the approach wherever noise mitigation or removal is implemented.

In [1]:
import torch
random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)
device = 'cuda:1' if torch.cuda.is_available() else 'cpu'
print('current device: ',device)

current device:  cuda:1


### Importing libraries required for code execution.

In [3]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
%matplotlib inline

import numpy as np
from PIL import Image

### Data Augmentation(My Own)


In [ ]:
transforms.RandomHorizontalFlip(0.5).forward()
transforms.ColorJitter().forward()
transforms.RandomRotation().forward()


### Code for loading the provided noisy dataset.
- note that basic augmentations (in the preprocessing stage) have already been applied.

In [4]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, pt_file):
        data = torch.load(pt_file)
        self.images = data['images']   
        self.targets = data['targets']
        
    def __len__(self):
        return len(self.targets)
    
    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        return image, target

In [5]:
def my_collate_fn(samples):
    images = []
    labels = []
    
    for data in samples:
        img, target = data
        images.append(img)
        labels.append(target)
        
    return torch.stack(images), torch.stack(labels)

### Data loading.
- "pt_file" referes to the location of the provided data file.
- the number of training samples is 43,201.

In [6]:
pt_file = os.path.join('Noisy_dataset.pt')

image_size = 64

train_dataset = CustomDataset(pt_file)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4, collate_fn=my_collate_fn)

print(f'Length of train samples: {len(train_dataset)}')

Length of train samples: 43201


In [7]:
print(  train_dataset.targets)


[tensor([97., -3., -3.]), tensor([97., -3., -3.]), tensor([97., -3., -3.]), tensor([97., -3., -3.]), tensor([97., -3., -3.]), tensor([97., -3., -3.]), tensor([88., -3., -3.]), tensor([26., -3., -3.]), tensor([63., -3., -3.]), tensor([88., -3., -3.]), tensor([26., -3., -3.]), tensor([26., -3., -3.]), tensor([26., -3., -3.]), tensor([26., -3., -3.]), tensor([26., -3., -3.]), tensor([63., -3., -3.]), tensor([ 3., -3., -3.]), tensor([26., -3., -3.]), tensor([97., -3., -3.]), tensor([97., -3., -3.]), tensor([ 3., -3., -3.]), tensor([26., -3., -3.]), tensor([26., -3., -3.]), tensor([50., -3., -3.]), tensor([50., -3., -3.]), tensor([50., -3., -3.]), tensor([63., -3., -3.]), tensor([88., -3., -3.]), tensor([ 3., -3., -3.]), tensor([ 3., -3., -3.]), tensor([ 3., -3., -3.]), tensor([26., -3., -3.]), tensor([26., -3., -3.]), tensor([63., -3., -3.]), tensor([63., -3., -3.]), tensor([90., -3., -3.]), tensor([ 3., -3., -3.]), tensor([23., -3., -3.]), tensor([26., -3., -3.]), tensor([26., -3., -3.]),

### Prepare the model; ResNet
- fill in the blank.
- ResNet-10 must be used without exception.

In [8]:
# Make the Basic Block of ResNet

class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, in_planes, planes, stride=1):    
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_planes, out_channels=planes, kernel_size=3, stride=stride, bias=False, padding=1)
        self.bn1 = nn.BatchNorm2d(planes)

        self.conv2 = nn.Conv2d(in_channels=planes, out_channels=planes, kernel_size=3, stride=1, bias=False, padding=1)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        # Fill in; Conditional statement related to the stride.
        #이 부분 검토하기
        if stride != 1 or in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels=in_planes,out_channels= planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes))

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [9]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=100):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=2)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

def ResNet10():
    return ResNet(BasicBlock, [1, 1, 1, 1])

In [10]:
# Some code for saving and loading model.
# Use if you needed.
def save_model(model, save_name):
    path = './' + str(save_name) + '.pt'
    ckpt = {'model': model}
    torch.save(ckpt, path)

def load_model(init_model, load_name):
    path = './' + str(load_name) + '.pt'
    #load_file = torch.load(path, map_location='cpu', weights_only=False)
    load_file = torch.load(path, map_location='cpu',weights_only=False)
    try:
        model = load_file['model']
        init_model.load_state_dict(model.state_dict())
    except:
        model = torch.load(str(load_name)+'.pt', weights_only=False)
        init_model = model
    return init_model

#### Implementation of the loss function for training.

In [11]:
# cross entropy function
def implemented_cross_entropy(output, target):
    if target.dim() == 2:
        target = target[:, 0]  # (batch_size,)
    
    target = target.long()
    
    log_probs = F.log_softmax(output, dim=1)  # (batch_size, num_classes)
    loss = -log_probs[torch.arange(output.size(0)), target]
    return loss.mean()

### Train function

In [12]:
def train(total_epoch, network, built_in_criterion, implemented_cross_entropy, optimizer, lr_schedule, train_loader, train_fn, device = 'cpu', save_name = 'save_name'):
    network = network.to(device)
    for epoch in range(total_epoch):
        train_fn(epoch, network, built_in_criterion, implemented_cross_entropy, optimizer, train_loader, device)
        lr_schedule.step()
        if ((epoch + 1) % 10 == 0) or epoch == total_epoch - 1:
            save_model(network, save_name)
            print('Model saved at epoch {} with name {} '.format(epoch + 1, save_name + '.pt'))
                

In [13]:
def train_single_epoch(current_epoch, network, built_in_criterion, implemented_cross_entropy, optimizer, train_loader, device='cpu'):
    network.train()
    running_loss = 0.0
    loss_error = 0.0
    correct, total_sample = 0.0, 0.0
    for idx, (input, label) in enumerate(train_loader):
        input, label = input.to(device), label.to(device)
        optimizer.zero_grad()
        output = network(input)

        _, pred = torch.max(output.data, 1)
        correct += (pred == label[:, 0].long()).sum().item() # The first index in the label represents the class ID. The remaining indices may be useful for the project.
        total_sample += label.size(0)

        loss = implemented_cross_entropy(output, label[:, 0].view(-1).long())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # calculate error between imported loss and implmented from scratch
        loss_error += torch.abs(built_in_criterion(output, label[:, 0].view(-1).long()).detach() - implemented_cross_entropy(output, label[:, 0].view(-1).long()).detach())

    print('Epoch: {} | Training Accuracy: {:.2f} % | Loss: {:.2f} | Loss error {:.5f}'.format(current_epoch, 100*correct/total_sample, running_loss/(idx+1), loss_error/(idx + 1)))

### Training begins!

In [14]:
network = ResNet10()
total_epoch = 100
built_in_criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
step_lr_scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[50, 75], gamma =0.1)

train_fn = train_single_epoch
train(total_epoch, network, built_in_criterion, implemented_cross_entropy, optimizer, step_lr_scheduler, train_loader, train_fn, 
device = device, save_name= f'김두회_20202927')

Epoch: 0 | Training Accuracy: 9.53 % | Loss: 4.07 | Loss error 0.00000
Epoch: 1 | Training Accuracy: 18.71 % | Loss: 3.58 | Loss error 0.00000
Epoch: 2 | Training Accuracy: 25.07 % | Loss: 3.26 | Loss error 0.00000
Epoch: 3 | Training Accuracy: 30.20 % | Loss: 3.01 | Loss error 0.00000
Epoch: 4 | Training Accuracy: 35.34 % | Loss: 2.77 | Loss error 0.00000
Epoch: 5 | Training Accuracy: 40.47 % | Loss: 2.53 | Loss error 0.00000
Epoch: 6 | Training Accuracy: 46.55 % | Loss: 2.24 | Loss error 0.00000
Epoch: 7 | Training Accuracy: 54.27 % | Loss: 1.90 | Loss error 0.00000
Epoch: 8 | Training Accuracy: 63.68 % | Loss: 1.49 | Loss error 0.00000
Epoch: 9 | Training Accuracy: 74.67 % | Loss: 1.06 | Loss error 0.00000
Model saved at epoch 10 with name 김두회_20202927.pt 
Epoch: 10 | Training Accuracy: 85.05 % | Loss: 0.66 | Loss error 0.00000
Epoch: 11 | Training Accuracy: 94.18 % | Loss: 0.31 | Loss error 0.00000
Epoch: 12 | Training Accuracy: 98.87 % | Loss: 0.11 | Loss error 0.00000
Epoch: 13 |

### Critical note regarding the testing process.
- the submitted models will be evaluated using the code that includes the test function below.
- make sure that the trained model runs inference properly with this code 
- since the test data is not provided, verification can be done using the train data.
- failure to run will be treated as an error.
- Name the .pt file as: Name_StudentID.pt.

In [16]:
def test(network, implemented_cross_entropy, test_loader, device = 'cpu', load_name = None):
    if load_name is not None:
        network = load_model(network, load_name)

    print('Test start')
    network = network.to(device)
    network.eval()
    test_loss = 0.0
    correct, total_sample = 0.0, 0.0
    with torch.no_grad():
        for idx, (image, label) in enumerate(test_loader):
            image, label = image.to(device), label.to(device)
            output = network(image)
            loss = implemented_cross_entropy(output, label[:, 0].view(-1).long())
            _, pred = torch.max(output.data, 1)
            print('Prediction values: {}' .format(pred))
            total_sample += label.size(0)
            correct += (pred == label[:, 0].long()).sum().item()
            test_loss += loss.item()
    print('Test loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(test_loss/(idx + 1) ,correct, total_sample, 100 * correct / total_sample))
    
network = ResNet10()
test(network, implemented_cross_entropy, train_loader, device = device, load_name= '김두회_20202927')

Test start
Prediction values: tensor([11, 16, 34, 10, 46, 51, 35, 19, 93, 13, 59, 66, 46, 11, 14, 58, 38, 34,
        30, 24, 22, 54, 71, 59, 52, 44, 66, 29, 43, 82, 32, 49, 47,  6,  6, 31,
        11, 80, 47, 42, 10, 32, 41, 65, 69, 20, 14, 31,  3, 66, 40, 46, 63, 19,
        11, 73, 82, 32, 74, 27, 47, 91, 67, 34, 13, 76, 53, 44, 11, 16, 71, 74,
        68, 39, 17, 46, 63, 55, 45, 34, 72, 80, 48, 76, 54, 48, 74, 44, 11, 24,
        68, 73, 49,  3, 80, 67, 23, 57, 37, 64, 16, 25, 33, 40, 14, 19, 51, 12,
        16, 16, 56, 44, 12, 23, 29,  8, 57, 53, 16,  2, 76, 65, 60,  9, 74,  8,
        15, 33], device='cuda:1')
Prediction values: tensor([71,  1, 31, 50, 17, 33, 69, 15, 89, 38, 79, 19, 55, 43, 57,  7, 78, 80,
        52, 65,  0, 56, 36, 14, 28, 77, 56, 35, 78,  7, 79, 26, 69, 54, 10, 35,
        60, 89, 69, 50, 30, 37, 10, 74, 80, 49, 18, 10, 85, 98, 32, 81, 64, 34,
        29,  9, 46, 75, 11, 71, 23, 66, 45, 71, 11, 62,  8, 10,  8, 20, 32, 38,
        52, 54,  7, 30,  7, 58, 34, 3